In [1]:
import tensorflow as tf
import numpy as np
import torch
import random
import pandas as pd
import os, sys
import time
import datetime
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

# random.seed(42)
torch.manual_seed(42) 
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: Tesla V100-SXM3-32GB


########################### Load dataset ###########################

In [2]:
all_attribute = open("attrribute_names_lists/all_attributes.txt", "r") 
data = all_attribute.read()
all_attribute_list = data.split("\n")
print((all_attribute_list))
all_attribute.close()

['countrywoman', 'sororal', 'witches', 'maidservant', 'mothers', 'diva', 'actress', 'spinster', 'mama', 'duchesses', 'barwoman', 'countrywomen', 'dowry', 'hostesses', 'airwomen', 'menopause', 'clitoris', 'princess', 'governesses', 'abbess', 'women', 'widow', 'ladies', 'sorceresses', 'madam', 'brides', 'baroness', 'housewives', 'godesses', 'niece', 'widows', 'lady', 'sister', 'brides', 'nun', 'adultresses', 'obstetrics', 'bellgirls', 'her', 'marchioness', 'princesses', 'empresses', 'mare', 'chairwoman', 'convent', 'priestesses', 'girlhood', 'ladies', 'queen', 'gals', 'mommies', 'maid', 'female_ejaculation', 'spokeswoman', 'seamstress', 'cowgirls', 'chick', 'spinsters', 'hair_salon', 'empress', 'mommy', 'feminism', 'gals', 'enchantress', 'gal', 'motherhood', 'estrogen', 'camerawomen', 'godmother', 'strongwoman', 'goddess', 'matriarch', 'aunt', 'chairwomen', "ma'am", 'sisterhood', 'hostess', 'estradiol', 'wife', 'mom', 'stewardess', 'females', 'viagra', 'spokeswomen', 'ma', 'belle', 'minx

In [3]:
# female_names_list
all_attribute = open("attrribute_names_lists/female_names.txt", "r") 
data = all_attribute.read()
female_names_list = data.split("\n")
print((female_names_list))
all_attribute.close()

['Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi', 'Abbie', 'Abby', 'Abigael', 'Abigail', 'Abigale', 'Abra', 'Acacia', 'Ada', 'Adah', 'Adaline', 'Adara', 'Addie', 'Addis', 'Adel', 'Adela', 'Adelaide', 'Adele', 'Adelice', 'Adelina', 'Adelind', 'Adeline', 'Adella', 'Adelle', 'Adena', 'Adey', 'Adi', 'Adiana', 'Adina', 'Adora', 'Adore', 'Adoree', 'Adorne', 'Adrea', 'Adria', 'Adriaens', 'Adrian', 'Adriana', 'Adriane', 'Adrianna', 'Adrianne', 'Adrien', 'Adriena', 'Adrienne', 'Aeriel', 'Aeriela', 'Aeriell', 'Ag', 'Agace', 'Agata', 'Agatha', 'Agathe', 'Aggi', 'Aggie', 'Aggy', 'Agna', 'Agnella', 'Agnes', 'Agnese', 'Agnesse', 'Agneta', 'Agnola', 'Agretha', 'Aida', 'Aidan', 'Aigneis', 'Aila', 'Aile', 'Ailee', 'Aileen', 'Ailene', 'Ailey', 'Aili', 'Ailina', 'Ailyn', 'Aime', 'Aimee', 'Aimil', 'Aina', 'Aindrea', 'Ainslee', 'Ainsley', 'Ainslie', 'Ajay', 'Alaine', 'Alameda', 'Alana', 'Alanah', 'Alane', 'Alanna', 'Alayne', 'Alberta', 'Albertina', 'Albertine', 'Albina', 'Alecia', 'Aleda', 'Aleece', 'Aleecia

In [4]:
# male_names_list
all_attribute = open("attrribute_names_lists/male_names.txt", "r") 
data = all_attribute.read()
male_names_list = data.split("\n")
print((male_names_list))
all_attribute.close()

['Aamir', 'Aaron', 'Abbey', 'Abbie', 'Abbot', 'Abbott', 'Abby', 'Abdel', 'Abdul', 'Abdulkarim', 'Abdullah', 'Abe', 'Abel', 'Abelard', 'Abner', 'Abraham', 'Abram', 'Ace', 'Adair', 'Adam', 'Adams', 'Addie', 'Adger', 'Aditya', 'Adlai', 'Adnan', 'Adolf', 'Adolfo', 'Adolph', 'Adolphe', 'Adolpho', 'Adolphus', 'Adrian', 'Adrick', 'Adrien', 'Agamemnon', 'Aguinaldo', 'Aguste', 'Agustin', 'Aharon', 'Ahmad', 'Ahmed', 'Ahmet', 'Ajai', 'Ajay', 'Al', 'Alaa', 'Alain', 'Alan', 'Alasdair', 'Alastair', 'Albatros', 'Albert', 'Alberto', 'Albrecht', 'Alden', 'Aldis', 'Aldo', 'Aldric', 'Aldrich', 'Aldus', 'Aldwin', 'Alec', 'Aleck', 'Alejandro', 'Aleks', 'Aleksandrs', 'Alessandro', 'Alex', 'Alexander', 'Alexei', 'Alexis', 'Alf', 'Alfie', 'Alfonse', 'Alfonso', 'Alfonzo', 'Alford', 'Alfred', 'Alfredo', 'Algernon', 'Ali', 'Alic', 'Alister', 'Alix', 'Allah', 'Allan', 'Allen', 'Alley', 'Allie', 'Allin', 'Allyn', 'Alonso', 'Alonzo', 'Aloysius', 'Alphonse', 'Alphonso', 'Alston', 'Alton', 'Alvin', 'Alwin', 'Amadeus'

In [5]:
# len(get_attributes_ids(all_attribute_list))

In [6]:
import job_data_unbalanced_original_and_flipped
# import job_data_unbalanced_test
df = job_data_unbalanced_original_and_flipped.final_bios_df
# test_df = job_data_unbalanced_test.test_final_bios_df
df

/project_ghent/bias_mitigation_in_text_classification/bias_in_bios_k_fold/job_data_unbalanced_original_and_flipped.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female_bios_df["job_cat"] = "female_job"
/project_ghent/bias_mitigation_in_text_classification/bias_in_bios_k_fold/job_data_unbalanced_original_and_flipped.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female_bios_df["job_cat_id"] = int(0)
/project_ghent/bias_mitigation_in_text_classification/bias_in_bios_k_fold/job_data_unbalanced_ori

,raw_title,gender,label,scrubbed,bio,flipped_bio,swapped,prof,job_cat,job_cat_id
31289,architect,M,1,"_ focuses on cloud security, identity and acce...","He focuses on cloud security, identity and acc...","She focuses on cloud security, identity and ac...","She focuses on cloud security, identity and ac...",architect,male_job,1
61990,software engineer,M,24,"_ is the creator of Statoshi, a fork of Bitcoi...","He is the creator of Statoshi, a fork of Bitco...","She is the creator of Statoshi, a fork of Bitc...","She is the creator of Statoshi, a fork of Bitc...",software_engineer,male_job,1
29313,comedian,M,4,"_ children's books include ""Pouch Potato, Char...","His children's books include ""Pouch Potato, Ch...","Her children's books include ""Pouch Potato, Sh...","Her children's books include ""Pouch Potato, Ch...",comedian,male_job,1
61302,psychologist,M,22,"_ also has a therapy practice, through which _...","He also has a therapy practice, through which ...","She also has a therapy practice, through which...","She also has a therapy practice, through which...",psychologist,female_job,0
60333,architect,M,1,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,architect,male_job,1
...,...,...,...,...,...,...,...,...,...,...
5098,DJ,M,8,"_’s toured the UK, Europe and Canada as well a...","He’s toured the UK, Europe and Canada as well ...","She’s toured the UK, Europe and Canada as well...","She’s toured the UK, Europe and Canada as well...",dj,male_job,1
11063,Surgeon,M,25,"Dr. _ graduated from Andhra Medical College, A...",Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,surgeon,male_job,1
49107,psychologist,F,22,_ has a longstanding interest in youth mental ...,She has a longstanding interest in youth menta...,He has a longstanding interest in youth mental...,He has a longstanding interest in youth mental...,psychologist,female_job,0
8028,Dietitian,F,7,_ provides one-on-one dietary counseling on we...,She provides one-on-one dietary counseling on ...,He provides one-on-one dietary counseling on w...,He provides one-on-one dietary counseling on w...,dietitian,female_job,0


In [7]:
df.rename(columns={'label':'old_label'}, inplace=True)
df

,raw_title,gender,old_label,scrubbed,bio,flipped_bio,swapped,prof,job_cat,job_cat_id
31289,architect,M,1,"_ focuses on cloud security, identity and acce...","He focuses on cloud security, identity and acc...","She focuses on cloud security, identity and ac...","She focuses on cloud security, identity and ac...",architect,male_job,1
61990,software engineer,M,24,"_ is the creator of Statoshi, a fork of Bitcoi...","He is the creator of Statoshi, a fork of Bitco...","She is the creator of Statoshi, a fork of Bitc...","She is the creator of Statoshi, a fork of Bitc...",software_engineer,male_job,1
29313,comedian,M,4,"_ children's books include ""Pouch Potato, Char...","His children's books include ""Pouch Potato, Ch...","Her children's books include ""Pouch Potato, Sh...","Her children's books include ""Pouch Potato, Ch...",comedian,male_job,1
61302,psychologist,M,22,"_ also has a therapy practice, through which _...","He also has a therapy practice, through which ...","She also has a therapy practice, through which...","She also has a therapy practice, through which...",psychologist,female_job,0
60333,architect,M,1,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,architect,male_job,1
...,...,...,...,...,...,...,...,...,...,...
5098,DJ,M,8,"_’s toured the UK, Europe and Canada as well a...","He’s toured the UK, Europe and Canada as well ...","She’s toured the UK, Europe and Canada as well...","She’s toured the UK, Europe and Canada as well...",dj,male_job,1
11063,Surgeon,M,25,"Dr. _ graduated from Andhra Medical College, A...",Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,surgeon,male_job,1
49107,psychologist,F,22,_ has a longstanding interest in youth mental ...,She has a longstanding interest in youth menta...,He has a longstanding interest in youth mental...,He has a longstanding interest in youth mental...,psychologist,female_job,0
8028,Dietitian,F,7,_ provides one-on-one dietary counseling on we...,She provides one-on-one dietary counseling on ...,He provides one-on-one dietary counseling on w...,He provides one-on-one dietary counseling on w...,dietitian,female_job,0


In [8]:
# df['label'] = 0
# for k,i in enumerate(df.old_label.unique()):
#     df['label'] = np.where(df['old_label']== i, k, df['label'])
# df

In [9]:
# label_dict = pd.Series(df.label.values,index=df.prof).to_dict()
# label_dict
label_dict = {
    'yoga_teacher': 0,
 'dietitian': 1,
 'comedian': 2,
 'composer': 3,
 'surgeon': 4,
 'nurse': 5,
 'rapper': 6,
 'dj': 7,
 'psychologist': 8,
 'model': 9,
 'interior_designer': 10,
 'software_engineer': 11,
 'paralegal': 12,
 'architect': 13
}

In [10]:
df.rename(columns={'label':'old_label'}, inplace=True)
df['label'] = df['prof'].map(label_dict)
df

,raw_title,gender,old_label,scrubbed,bio,flipped_bio,swapped,prof,job_cat,job_cat_id,label
31289,architect,M,1,"_ focuses on cloud security, identity and acce...","He focuses on cloud security, identity and acc...","She focuses on cloud security, identity and ac...","She focuses on cloud security, identity and ac...",architect,male_job,1,13
61990,software engineer,M,24,"_ is the creator of Statoshi, a fork of Bitcoi...","He is the creator of Statoshi, a fork of Bitco...","She is the creator of Statoshi, a fork of Bitc...","She is the creator of Statoshi, a fork of Bitc...",software_engineer,male_job,1,11
29313,comedian,M,4,"_ children's books include ""Pouch Potato, Char...","His children's books include ""Pouch Potato, Ch...","Her children's books include ""Pouch Potato, Sh...","Her children's books include ""Pouch Potato, Ch...",comedian,male_job,1,2
61302,psychologist,M,22,"_ also has a therapy practice, through which _...","He also has a therapy practice, through which ...","She also has a therapy practice, through which...","She also has a therapy practice, through which...",psychologist,female_job,0,8
60333,architect,M,1,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,architect,male_job,1,13
...,...,...,...,...,...,...,...,...,...,...,...
5098,DJ,M,8,"_’s toured the UK, Europe and Canada as well a...","He’s toured the UK, Europe and Canada as well ...","She’s toured the UK, Europe and Canada as well...","She’s toured the UK, Europe and Canada as well...",dj,male_job,1,7
11063,Surgeon,M,25,"Dr. _ graduated from Andhra Medical College, A...",Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,surgeon,male_job,1,4
49107,psychologist,F,22,_ has a longstanding interest in youth mental ...,She has a longstanding interest in youth menta...,He has a longstanding interest in youth mental...,He has a longstanding interest in youth mental...,psychologist,female_job,0,8
8028,Dietitian,F,7,_ provides one-on-one dietary counseling on we...,She provides one-on-one dietary counseling on ...,He provides one-on-one dietary counseling on w...,He provides one-on-one dietary counseling on w...,dietitian,female_job,0,1


In [11]:
df.label.value_counts().to_dict()

{8: 4151,
 13: 2796,
 5: 2788,
 4: 2736,
 11: 1990,
 3: 1750,
 9: 1549,
 1: 1006,
 2: 813,
 12: 412,
 0: 406,
 10: 393,
 6: 362,
 7: 347}

In [12]:
# df.rename(columns={'label':'old_label'}, inplace=True)
# df

In [13]:
# df['label'] = 0
# for k,i in enumerate(df.old_label.unique()):
#     df['label'] = np.where(df['old_label']== i, k, df['label'])
# df

In [14]:
gender = {'M': 1,'F': 0}
df.gender = [gender[item] for item in df.gender]
df

,raw_title,gender,old_label,scrubbed,bio,flipped_bio,swapped,prof,job_cat,job_cat_id,label
31289,architect,1,1,"_ focuses on cloud security, identity and acce...","He focuses on cloud security, identity and acc...","She focuses on cloud security, identity and ac...","She focuses on cloud security, identity and ac...",architect,male_job,1,13
61990,software engineer,1,24,"_ is the creator of Statoshi, a fork of Bitcoi...","He is the creator of Statoshi, a fork of Bitco...","She is the creator of Statoshi, a fork of Bitc...","She is the creator of Statoshi, a fork of Bitc...",software_engineer,male_job,1,11
29313,comedian,1,4,"_ children's books include ""Pouch Potato, Char...","His children's books include ""Pouch Potato, Ch...","Her children's books include ""Pouch Potato, Sh...","Her children's books include ""Pouch Potato, Ch...",comedian,male_job,1,2
61302,psychologist,1,22,"_ also has a therapy practice, through which _...","He also has a therapy practice, through which ...","She also has a therapy practice, through which...","She also has a therapy practice, through which...",psychologist,female_job,0,8
60333,architect,1,1,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,architect,male_job,1,13
...,...,...,...,...,...,...,...,...,...,...,...
5098,DJ,1,8,"_’s toured the UK, Europe and Canada as well a...","He’s toured the UK, Europe and Canada as well ...","She’s toured the UK, Europe and Canada as well...","She’s toured the UK, Europe and Canada as well...",dj,male_job,1,7
11063,Surgeon,1,25,"Dr. _ graduated from Andhra Medical College, A...",Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,surgeon,male_job,1,4
49107,psychologist,0,22,_ has a longstanding interest in youth mental ...,She has a longstanding interest in youth menta...,He has a longstanding interest in youth mental...,He has a longstanding interest in youth mental...,psychologist,female_job,0,8
8028,Dietitian,0,7,_ provides one-on-one dietary counseling on we...,She provides one-on-one dietary counseling on ...,He provides one-on-one dietary counseling on w...,He provides one-on-one dietary counseling on w...,dietitian,female_job,0,1


In [15]:
# import pandas as pd
import nltk
nltk.download('punkt')

df['tokenized_scrubbed'] = df.apply(lambda row: nltk.word_tokenize(row['scrubbed']), axis=1)
df

[nltk_data] Downloading package punkt to /home/etokpoua/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,raw_title,gender,old_label,scrubbed,bio,flipped_bio,swapped,prof,job_cat,job_cat_id,label,tokenized_scrubbed
31289,architect,1,1,"_ focuses on cloud security, identity and acce...","He focuses on cloud security, identity and acc...","She focuses on cloud security, identity and ac...","She focuses on cloud security, identity and ac...",architect,male_job,1,13,"[_, focuses, on, cloud, security, ,, identity,..."
61990,software engineer,1,24,"_ is the creator of Statoshi, a fork of Bitcoi...","He is the creator of Statoshi, a fork of Bitco...","She is the creator of Statoshi, a fork of Bitc...","She is the creator of Statoshi, a fork of Bitc...",software_engineer,male_job,1,11,"[_, is, the, creator, of, Statoshi, ,, a, fork..."
29313,comedian,1,4,"_ children's books include ""Pouch Potato, Char...","His children's books include ""Pouch Potato, Ch...","Her children's books include ""Pouch Potato, Sh...","Her children's books include ""Pouch Potato, Ch...",comedian,male_job,1,2,"[_, children, 's, books, include, ``, Pouch, P..."
61302,psychologist,1,22,"_ also has a therapy practice, through which _...","He also has a therapy practice, through which ...","She also has a therapy practice, through which...","She also has a therapy practice, through which...",psychologist,female_job,0,8,"[_, also, has, a, therapy, practice, ,, throug..."
60333,architect,1,1,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,architect,male_job,1,13,"[The, exhibition, explores, concrete, construc..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5098,DJ,1,8,"_’s toured the UK, Europe and Canada as well a...","He’s toured the UK, Europe and Canada as well ...","She’s toured the UK, Europe and Canada as well...","She’s toured the UK, Europe and Canada as well...",dj,male_job,1,7,"[_, ’, s, toured, the, UK, ,, Europe, and, Can..."
11063,Surgeon,1,25,"Dr. _ graduated from Andhra Medical College, A...",Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,surgeon,male_job,1,4,"[Dr., _, graduated, from, Andhra, Medical, Col..."
49107,psychologist,0,22,_ has a longstanding interest in youth mental ...,She has a longstanding interest in youth menta...,He has a longstanding interest in youth mental...,He has a longstanding interest in youth mental...,psychologist,female_job,0,8,"[_, has, a, longstanding, interest, in, youth,..."
8028,Dietitian,0,7,_ provides one-on-one dietary counseling on we...,She provides one-on-one dietary counseling on ...,He provides one-on-one dietary counseling on w...,He provides one-on-one dietary counseling on w...,dietitian,female_job,0,1,"[_, provides, one-on-one, dietary, counseling,..."


In [16]:
all_names = male_names_list + female_names_list
# all_names

In [17]:
lower_all_attribute_list = [i.lower() for i in all_attribute_list]
all_attribute_list

['countrywoman',
 'sororal',
 'witches',
 'maidservant',
 'mothers',
 'diva',
 'actress',
 'spinster',
 'mama',
 'duchesses',
 'barwoman',
 'countrywomen',
 'dowry',
 'hostesses',
 'airwomen',
 'menopause',
 'clitoris',
 'princess',
 'governesses',
 'abbess',
 'women',
 'widow',
 'ladies',
 'sorceresses',
 'madam',
 'brides',
 'baroness',
 'housewives',
 'godesses',
 'niece',
 'widows',
 'lady',
 'sister',
 'brides',
 'nun',
 'adultresses',
 'obstetrics',
 'bellgirls',
 'her',
 'marchioness',
 'princesses',
 'empresses',
 'mare',
 'chairwoman',
 'convent',
 'priestesses',
 'girlhood',
 'ladies',
 'queen',
 'gals',
 'mommies',
 'maid',
 'female_ejaculation',
 'spokeswoman',
 'seamstress',
 'cowgirls',
 'chick',
 'spinsters',
 'hair_salon',
 'empress',
 'mommy',
 'feminism',
 'gals',
 'enchantress',
 'gal',
 'motherhood',
 'estrogen',
 'camerawomen',
 'godmother',
 'strongwoman',
 'goddess',
 'matriarch',
 'aunt',
 'chairwomen',
 "ma'am",
 'sisterhood',
 'hostess',
 'estradiol',
 'wife',

In [18]:
def replace_tokens(wordlist):
    lower_wordlist = [i.lower() for i in wordlist]
    for i in range(len(wordlist)):
        if lower_wordlist[i] in all_attribute_list:
            wordlist[i] = "_"
        if wordlist[i] in all_names:
            wordlist[i] = "_"
    return wordlist

In [19]:
df['tokenized_scrubbed'] = df.apply(lambda row: replace_tokens(row['tokenized_scrubbed']), axis=1)
df

,raw_title,gender,old_label,scrubbed,bio,flipped_bio,swapped,prof,job_cat,job_cat_id,label,tokenized_scrubbed
31289,architect,1,1,"_ focuses on cloud security, identity and acce...","He focuses on cloud security, identity and acc...","She focuses on cloud security, identity and ac...","She focuses on cloud security, identity and ac...",architect,male_job,1,13,"[_, focuses, on, cloud, security, ,, identity,..."
61990,software engineer,1,24,"_ is the creator of Statoshi, a fork of Bitcoi...","He is the creator of Statoshi, a fork of Bitco...","She is the creator of Statoshi, a fork of Bitc...","She is the creator of Statoshi, a fork of Bitc...",software_engineer,male_job,1,11,"[_, is, the, creator, of, Statoshi, ,, a, fork..."
29313,comedian,1,4,"_ children's books include ""Pouch Potato, Char...","His children's books include ""Pouch Potato, Ch...","Her children's books include ""Pouch Potato, Sh...","Her children's books include ""Pouch Potato, Ch...",comedian,male_job,1,2,"[_, children, 's, books, include, ``, Pouch, P..."
61302,psychologist,1,22,"_ also has a therapy practice, through which _...","He also has a therapy practice, through which ...","She also has a therapy practice, through which...","She also has a therapy practice, through which...",psychologist,female_job,0,8,"[_, also, has, a, therapy, practice, ,, throug..."
60333,architect,1,1,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,architect,male_job,1,13,"[The, exhibition, explores, concrete, construc..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5098,DJ,1,8,"_’s toured the UK, Europe and Canada as well a...","He’s toured the UK, Europe and Canada as well ...","She’s toured the UK, Europe and Canada as well...","She’s toured the UK, Europe and Canada as well...",dj,male_job,1,7,"[_, ’, s, toured, the, UK, ,, Europe, and, _, ..."
11063,Surgeon,1,25,"Dr. _ graduated from Andhra Medical College, A...",Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,surgeon,male_job,1,4,"[Dr., _, graduated, from, Andhra, Medical, Col..."
49107,psychologist,0,22,_ has a longstanding interest in youth mental ...,She has a longstanding interest in youth menta...,He has a longstanding interest in youth mental...,He has a longstanding interest in youth mental...,psychologist,female_job,0,8,"[_, has, a, longstanding, interest, in, youth,..."
8028,Dietitian,0,7,_ provides one-on-one dietary counseling on we...,She provides one-on-one dietary counseling on ...,He provides one-on-one dietary counseling on w...,He provides one-on-one dietary counseling on w...,dietitian,female_job,0,1,"[_, provides, one-on-one, dietary, counseling,..."


In [20]:
df['scrubbed2'] = df.apply(lambda row: " ".join(row['tokenized_scrubbed']), axis=1)
# joined = " ".joi(list)
df

,raw_title,gender,old_label,scrubbed,bio,flipped_bio,swapped,prof,job_cat,job_cat_id,label,tokenized_scrubbed,scrubbed2
31289,architect,1,1,"_ focuses on cloud security, identity and acce...","He focuses on cloud security, identity and acc...","She focuses on cloud security, identity and ac...","She focuses on cloud security, identity and ac...",architect,male_job,1,13,"[_, focuses, on, cloud, security, ,, identity,...","_ focuses on cloud security , identity and acc..."
61990,software engineer,1,24,"_ is the creator of Statoshi, a fork of Bitcoi...","He is the creator of Statoshi, a fork of Bitco...","She is the creator of Statoshi, a fork of Bitc...","She is the creator of Statoshi, a fork of Bitc...",software_engineer,male_job,1,11,"[_, is, the, creator, of, Statoshi, ,, a, fork...","_ is the creator of Statoshi , a fork of Bitco..."
29313,comedian,1,4,"_ children's books include ""Pouch Potato, Char...","His children's books include ""Pouch Potato, Ch...","Her children's books include ""Pouch Potato, Sh...","Her children's books include ""Pouch Potato, Ch...",comedian,male_job,1,2,"[_, children, 's, books, include, ``, Pouch, P...","_ children 's books include `` Pouch Potato , ..."
61302,psychologist,1,22,"_ also has a therapy practice, through which _...","He also has a therapy practice, through which ...","She also has a therapy practice, through which...","She also has a therapy practice, through which...",psychologist,female_job,0,8,"[_, also, has, a, therapy, practice, ,, throug...","_ also has a therapy practice , through which ..."
60333,architect,1,1,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,The exhibition explores concrete construction ...,architect,male_job,1,13,"[The, exhibition, explores, concrete, construc...",The exhibition explores concrete construction ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5098,DJ,1,8,"_’s toured the UK, Europe and Canada as well a...","He’s toured the UK, Europe and Canada as well ...","She’s toured the UK, Europe and Canada as well...","She’s toured the UK, Europe and Canada as well...",dj,male_job,1,7,"[_, ’, s, toured, the, UK, ,, Europe, and, _, ...","_ ’ s toured the UK , Europe and _ as well as ..."
11063,Surgeon,1,25,"Dr. _ graduated from Andhra Medical College, A...",Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,Dr. Chacko graduated from Andhra Medical Colle...,surgeon,male_job,1,4,"[Dr., _, graduated, from, Andhra, Medical, Col...","Dr. _ graduated from Andhra Medical College , ..."
49107,psychologist,0,22,_ has a longstanding interest in youth mental ...,She has a longstanding interest in youth menta...,He has a longstanding interest in youth mental...,He has a longstanding interest in youth mental...,psychologist,female_job,0,8,"[_, has, a, longstanding, interest, in, youth,...",_ has a longstanding interest in youth mental ...
8028,Dietitian,0,7,_ provides one-on-one dietary counseling on we...,She provides one-on-one dietary counseling on ...,He provides one-on-one dietary counseling on w...,He provides one-on-one dietary counseling on w...,dietitian,female_job,0,1,"[_, provides, one-on-one, dietary, counseling,...",_ provides one-on-one dietary counseling on we...


In [21]:
k_fold_df = np.array_split(df, 10)


########################### tokenize dataset ###########################

In [22]:
# from transformers import BertTokenizer, AdamW, BertConfig, BertForPreTraining

from transformers import AlbertTokenizer
tokenizer = AlbertTokenizer.from_pretrained('albert-large-v2', do_lower_case=True)
seq_len = 100
vocab_length = 30522
def tokenize(data, y, g):
    

    # Load the BERT tokenizer.
    print('Loading BERT tokenizer...')

    # Training Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []

    # For every sentence...
    for k, sent in enumerate(data):
        encoded_dict = tokenizer.encode_plus(
                            str(sent),                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = seq_len,           # Pad & truncate all sentences.
                            truncation=True,
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )
        

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        
        
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    class_labels = torch.tensor(y)
    gender_labels = torch.tensor(g)
    
    
    


#     print(y)
#     print(y_onehot)

    return input_ids, class_labels, attention_masks, gender_labels



Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/685 [00:00<?, ?B/s]

########################### save dataset ###########################

In [23]:
for ind, i in enumerate(k_fold_df):
    sentences = i.bio.values
    gender = i.gender.values
    labels = i.label.values
    train_input_ids, train_class_labels, train_attention_masks, train_gender_labels = tokenize(sentences, labels, gender)
    # test_input_ids, test_class_labels, test_attention_masks, test_gender_labels = tokenize(X_test, y_test, gen_test)
#     val_input_ids, val_class_labels, val_attention_masks, val_gender_labels = tokenize(X_val, y_val, gen_val)

    print('Original: ', sentences[0])
    print('Token IDs:', train_input_ids[0])
    print('Token IDs:', train_class_labels[0])
    
    from torch.utils.data import TensorDataset

    train_dataset = TensorDataset(train_input_ids, train_class_labels, train_attention_masks, train_gender_labels)

    print('{:>5,} training samples'.format(len(train_dataset)))

    name = "bio_classification_train_dataset_" + str(ind)
    torch.save(train_dataset, 'unbalanced_datasets/'+name+'.pt')
    print(name)

Loading BERT tokenizer...


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  He focuses on cloud security, identity and access management, mobility security, and security for Microsoft platforms and solutions. Jan is based in Belgium.
Token IDs: tensor([    2,    24,  7155,    27,  4005,  1221,    15,  3270,    17,  1381,
         1097,    15, 14806,  1221,    15,    17,  1221,    26,  7099,  6843,
           17,  6776,     9,  2262,    25,   432,    19,  4692,     9,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
Token IDs: tensor(13)
2,150 training sa

In [24]:
for ind, i in enumerate(k_fold_df):
    sentences = i.scrubbed2.values
    gender = i.gender.values
    labels = i.label.values
    train_input_ids, train_class_labels, train_attention_masks, train_gender_labels = tokenize(sentences, labels, gender)
    # test_input_ids, test_class_labels, test_attention_masks, test_gender_labels = tokenize(X_test, y_test, gen_test)
#     val_input_ids, val_class_labels, val_attention_masks, val_gender_labels = tokenize(X_val, y_val, gen_val)

    print('Original: ', sentences[0])
    print('Token IDs:', train_input_ids[0])
    print('Token IDs:', train_class_labels[0])
    
    from torch.utils.data import TensorDataset

    train_dataset = TensorDataset(train_input_ids, train_class_labels, train_attention_masks, train_gender_labels)

    print('{:>5,} training samples'.format(len(train_dataset)))

    name = "scrubbed_classification_train_dataset_" + str(ind)
    torch.save(train_dataset, 'unbalanced_datasets/'+name+'.pt')
    print(name)

Loading BERT tokenizer...
Original:  _ focuses on cloud security , identity and access management , mobility security , and security for Microsoft platforms and solutions . _ is based in Belgium .
Token IDs: tensor([    2,    13,     1,  7155,    27,  4005,  1221,    13,    15,  3270,
           17,  1381,  1097,    13,    15, 14806,  1221,    13,    15,    17,
         1221,    26,  7099,  6843,    17,  6776,    13,     9,    13,     1,
           25,   432,    19,  4692,    13,     9,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
Token IDs: 

In [25]:
for ind, i in enumerate(k_fold_df):
    sentences = i.flipped_bio.values
    gender = i.gender.values
    labels = i.label.values
    train_input_ids, train_class_labels, train_attention_masks, train_gender_labels = tokenize(sentences, labels, gender)
    # test_input_ids, test_class_labels, test_attention_masks, test_gender_labels = tokenize(X_test, y_test, gen_test)
#     val_input_ids, val_class_labels, val_attention_masks, val_gender_labels = tokenize(X_val, y_val, gen_val)

    print('Original: ', sentences[0])
    print('Token IDs:', train_input_ids[0])
    print('Token IDs:', train_class_labels[0])
    
    from torch.utils.data import TensorDataset

    train_dataset = TensorDataset(train_input_ids, train_class_labels, train_attention_masks, train_gender_labels)

    print('{:>5,} training samples'.format(len(train_dataset)))

    name = "flipped_classification_train_dataset_" + str(ind)
    torch.save(train_dataset, 'unbalanced_datasets/'+name+'.pt')
    print(name)

Loading BERT tokenizer...
Original:  She focuses on cloud security, identity and access management, mobility security, and security for Microsoft platforms and solutions. Jan is based in Belgium.
Token IDs: tensor([    2,    39,  7155,    27,  4005,  1221,    15,  3270,    17,  1381,
         1097,    15, 14806,  1221,    15,    17,  1221,    26,  7099,  6843,
           17,  6776,     9,  2262,    25,   432,    19,  4692,     9,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
Token IDs: t

In [26]:
for ind, i in enumerate(k_fold_df):
    from sklearn.utils import shuffle
    df_1 =  i[["gender", "label", "bio"]]
    df_2 =  i[["gender", "label", "flipped_bio"]]
    df_2.rename(columns={'flipped_bio':'bio'}, inplace=True)
    dff = pd.concat([df_1, df_2])
    dff = shuffle(dff, random_state=42)
    sentences = dff.bio.values
    gender = dff.gender.values
    labels = dff.label.values
    train_input_ids, train_class_labels, train_attention_masks, train_gender_labels = tokenize(sentences, labels, gender)
    # test_input_ids, test_class_labels, test_attention_masks, test_gender_labels = tokenize(X_test, y_test, gen_test)
#     val_input_ids, val_class_labels, val_attention_masks, val_gender_labels = tokenize(X_val, y_val, gen_val)

    print('Original: ', sentences[0])
    print('Token IDs:', train_input_ids[0])
    print('Token IDs:', train_class_labels[0])
    
    from torch.utils.data import TensorDataset

    train_dataset = TensorDataset(train_input_ids, train_class_labels, train_attention_masks, train_gender_labels)

    print('{:>5,} training samples'.format(len(train_dataset)))

    name = "bio_and_flipped_classification_train_dataset_" + str(ind)
    torch.save(train_dataset, 'unbalanced_datasets/'+name+'.pt')
    print(name)

Loading BERT tokenizer...


/tmp/ipykernel_42992/1204574785.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.rename(columns={'flipped_bio':'bio'}, inplace=True)
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  She completed an AS at Mercer County Community College, a BA in Psychology at The College of New Jersey, and an MA in Applied Clinical Psychology at William Paterson University. She is currently pursing her BSN at Walden University.
Token IDs: tensor([    2,    39,  1066,    40,    28,    35, 14035,   271,   514,   314,
           15,    21,  2683,    19,  6182,    35,    14,   314,    16,    78,
         2134,    15,    17,    40,  1216,    19,  2435,  5611,  6182,    35,
          605, 22228,   155,     9,    39,    25,   871,  6771,    18,    68,
           36,   334,    18,   103,    35, 25043,   155,     9,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0, 

/tmp/ipykernel_42992/1204574785.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.rename(columns={'flipped_bio':'bio'}, inplace=True)
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  He studied architecture in Thessaloniki, Greece and he holds an MSc from Columbia University and a PhD from TU Delft. His thesis investigated the recent history of planning in Athens and the link between conflict, urban management and architectural form. His research interests explore architecture in relation to the politics of labour, law and social reform. Prior to the RCA, he taught at the Berlage Institute/Rotterdam and since 2012 at the MArch Urban Design at the Bartlett.
Token IDs: tensor([    2,    24,  1449,  2607,    19, 25201,    15,  4309,    17,    24,
         2763,    40,  4235,   150,    37,  2368,   155,    17,    21,  7739,
           37,  2289,  1506,  3072,     9,    33, 10452, 11790,    14,  1764,
          447,    16,  2334,    19,  7300,    17,    14,  3508,   128,  2930,
           15,  1980,  1097,    17,  5350,   505,     9,    33,   527,  4408,
         8594,  2607,    19,  5827,    20,    14,  2634,    16,  2668,    15,
          451,    17,   668,

...

Original:  His bad luck evaporates when meeting talented singing Chipmunks Alvin, Simon and Theodore. Soon they became a true musical sensation. Sweet animals do not fail to occur wild side of nature, so Dave is forced to use all their ingenuity to smooth out the problems and return his family before it's too late.
Token IDs: tensor([    2,    33,   896,  5419, 25098,    18,    76,  1235, 11847,  3385,
         7525,  5619,  4020, 19768,    15,  2214,    17, 10358,     9,   651,
           59,   178,    21,  1151,  1457,  7126,     9,  2092,  2522,   107,
           52,  7476,    20,  3744,  1808,   270,    16,  1444,    15,    86,
         3498,    25,  1292,    20,   275,    65,    66,    13, 10180,   291,
          856,    20,  3905,    70,    14,  1716,    17,   788,    33,   190,
          115,    32,    22,    18,   266,   456,     9,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0

/tmp/ipykernel_42992/1204574785.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.rename(columns={'flipped_bio':'bio'}, inplace=True)
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  She received her Bachelor of Science Degree in Nursing from Murray State University and her Masters of Science Degree in Nursing from the University of Kentucky. She brings with her over twenty-five years of experience in Family Medicine. In her spare time, Ms. Hunter enjoys spending time with her husband and two daughters.
Token IDs: tensor([    2,    39,   420,    36,  3427,    16,   762,  1168,    19,  8181,
           37,  5169,   146,   155,    17,    36,  5001,    16,   762,  1168,
           19,  8181,    37,    14,   155,    16,  4425,     9,    39,  7620,
           29,    36,    84,  1309,     8,  4709,   122,    16,  1496,    19,
          190,  2333,     9,    19,    36,  8001,    85,    15,  4235,     9,
         2944, 18431,  4837,    85,    29,    36,  1253,    17,    81,  4909,
            9,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,

/tmp/ipykernel_42992/1204574785.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.rename(columns={'flipped_bio':'bio'}, inplace=True)
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  He's also tnd member of the core Activiti development team. He's also the coauthor of Manning's "Open Source ESBs in Action." he coauthor of Manning's "Open Source ESBs in Action." See less
Token IDs: tensor([    2,    24,    22,    18,    67,    13,    38,   706,   322,    16,
           14,  2884,    13, 19516,  8793,   522,   173,     9,    24,    22,
           18,    67,    14,   326, 10007,    16, 12481,    22,    18,    13,
            7, 10157,  1267,    13,   160,  4562,    19,  1028,     9,     7,
           24,   326, 10007,    16, 12481,    22,    18,    13,     7, 10157,
         1267,    13,   160,  4562,    19,  1028,     9,     7,   196,   787,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
Token I